# Семинар 12. Анализ независимых компонент (ICA)

## План семинара:

  0. ICA (recap PCA);
  1. Сравнение реализаций ICA;
  2. ICA для анализа фМРТ данных;
  3. Вспомним задачу с болезнью Альцгеймера.

Источники:

* https://blog.paperspace.com/dimension-reduction-with-independent-components-analysis/
* https://towardsdatascience.com/independent-component-analysis-via-gradient-ascent-in-numpy-and-tensorflow-with-interactive-code-98b9a73e5d19
* https://github.com/asdspal/dimRed
* http://nilearn.github.io/auto_examples/05_advanced/plot_ica_resting_state.html
* https://nilearn.github.io/auto_examples/03_connectivity/plot_canica_resting_state.html#sphx-glr-auto-examples-03-connectivity-plot-canica-resting-state-py

Литература:

* Эз а маст: https://arxiv.org/abs/1711.10873
* https://www.sciencedirect.com/science/article/pii/S0893608000000265

## 0.1. ICA

ICA представляет собой метод линейного уменьшения размеров, который пытается восстановить независимые компоненты в сигнале. Так же ICA называют методом слепого разложения сигнала для «задачи коктейльной вечеринки». 


ICA является важным инструментом в анализе нейровизуализации, fMRI и EEG, который помогает отделить полезные сигналы от аномальных (шумов и артефактов). 

ICA предполагает, что каждый образец данных представляет собой смесь независимых компонент, и он стремится найти эти независимые компоненты. В основе ICA лежит «НЕЗАВИСИМОСТЬ». Мы должны сначала попытаться понять это.

Когда можно смело сказать, что две переменные независимы? Чем он отличается от «Корреляции»? И, наконец, как вы измеряете степень независимости?

##### Пример: какой уровень корреляции между этими двумя сигналами?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
plt.figure()
x = np.array([-5,-4,-3,-2,-1,0,1,2,3,4,5])
y = np.array([25,16,9,4,1,0,1,4,9,16,25])
plt.plot(x)
plt.plot(y)
plt.show()

In [ ]:
np.correlate(x,y)

##### Вопрос 1: почему Гауссовые переменные плохо разделяются с ICA?

Согласно Центральной предельной теореме https://en.wikipedia.org/wiki/Central_limit_theorem, сумма независимых случайных переменных ближе к гауссу чем каждая из величин сама по себе. 

Поэтому возникает интуиция, что чтобы восстановить исходные переменные, мы должны заложить требования «негауссовости» в них. 

В случае распределения Гаусса, независимые гауссовские переменные также независимы, это уникальное свойство, связанное с распределением Гаусса.

Давайте рассмотрим простой пример, чтобы понять эту концепцию. Посмотрим на четыре набора данных - два из гауссовского распределения и два из равномерного распределения.

##### Подход ICA к проблеме выделения независимых компонент основан на трех предположениях:

* процесс смешивания является линейным;
* все сигналы источника не зависят друг от друга;
* все исходные сигналы имеют не-гауссовское распределение.

##### Вопросы:

  * Сработает ли метод с одной гауссовой переменной(или 2)?
  * Какие переменные называются независимыми?

###### Почему важно что исходные сигналы имеют не-гауссовское распределение?

In [ ]:
np.random.seed(100)
U1 = np.random.uniform(-1, 1, 1000) # генерируем семплы для двумерного равномерного распределения
U2 = np.random.uniform(-1, 1, 1000)

G1 = np.random.randn(1000) # генерируем семплы для двумерного нормального распределения
G2 = np.random.randn(1000)

%matplotlib inline

from matplotlib import pyplot as plt

fig = plt.figure()

ax1 = fig.add_subplot(121, aspect = "equal")
ax1.scatter(U1, U2, marker = ".")
ax1.set_title("Uniform")


ax2 = fig.add_subplot(122, aspect = "equal")
ax2.scatter(G1, G2, marker = ".")
ax2.set_title("Gaussian")


plt.show()

In [ ]:
# возьмем случайную Матрицу А, но которую мы домножим наши распределения
A = np.array([[1, 0], [1, 2]])
A = np.array([[1 / np.sqrt(2), 1 / np.sqrt(2)], [1 / np.sqrt(2), - 1 / np.sqrt(2)]])

U_source = np.array([U1,U2])
U_mix = U_source.T.dot(A)

G_source = np.array([G1, G2])
G_mix = G_source.T.dot(A)


fig  = plt.figure()

ax1 = fig.add_subplot(121)
ax1.set_title("Mixed Uniform ")
ax1.scatter(U_mix[:, 0], U_mix[:,1], marker = ".")

ax2 = fig.add_subplot(122)
ax2.set_title("Mixed Gaussian ")
ax2.scatter(G_mix[:, 0], G_mix[:, 1], marker = ".")


plt.show()   

In [ ]:
# PCA с отбелением данных (whitening)
from sklearn.decomposition import PCA 
U_pca = PCA(whiten=True).fit_transform(U_mix)
G_pca = PCA(whiten=True).fit_transform(G_mix)

# let's plot the uncorrelated columns from the datasets
fig  = plt.figure()

ax1 = fig.add_subplot(121)
ax1.set_title("PCA Uniform ")
ax1.scatter(U_pca[:, 0], U_pca[:,1], marker = ".")

ax2 = fig.add_subplot(122)
ax2.set_title("PCA Gaussian ")
ax2.scatter(G_pca[:, 0], G_pca[:, 1], marker = ".")

Как видите, в случае гауссовых переменных мы потеряли информацию о матрице A!

#####  Вопрос 2: Как можно оценить меру негауссовости процесса?

Через kurtosis(коэффициент эксцесса) и негэнтропию: http://fourier.eng.hmc.edu/e161/lectures/ica/node4.html

Идея в том, что гаусс обладает самой высокой энтропией среди всех распределений с одинаковой дисперсией, поэтому если мы хотим найти негауссово распределение, то нам нужно добавить регуляризацию на его энтропию:

$$H(\mathcal{N}(0, \sigma^2)) - H(p(x))) \geq C$$

И интересной особенностью этого выражения является то что оно выражается через коэффициент эксцесса, т.е.

$$H(\mathcal{N}(0, \sigma^2)) - H(p(y))) \approx \frac{1}{12} \mathbb E[y^3]^2 + \frac{1}{48} kurt(x)^3 \geq C$$

Однако так как мы рассматриваем совместное распределение большого числа величин, то удобнее вести речь в терминах энтропий и взаимных информаций.

#### Избавляется от нелинейной зависимости

Мы можем минимизировать нелинейную зависимость, уменьшая взаимную информацию между переменными. Из теории информации взаимную информацию можно определить как:

<center>$MI$ = сумма энтропий маргинальных функций распределения - энтропия функции совместного распределения:<p>
    
Где энтропия - это показатель средней информации, содержащейся в данных. Для переменной $ x $ с распределением $p(x)$ она определяется как

$$ H(x) = - \sum\limits_{i}\,(\,p\,(x_i) \times \log_2 p(\,x_i\,))$$
    
![](http://fourier.eng.hmc.edu/e161/lectures/figures/mutual_info.gif)

Итак, для нахождения энтропии двух переменных $ x $, $ y $ мы должны минимизировать следующую функцию, где $z = p(x, y) $ совместная плотность распределения  $x$ и $y$:
    
$$MI = H(x)+ H(y) - H(z)$$


Предположим, что W - это матрица преобразования, которую нам нужно умножить на $X_{new}$, чтобы удалить зависимость более высокого порядка. Затем мы можем определить

$$MI = \sum\limits_{i}H(W_i\times X_{pca_{i}}) - H(W\times X_{pca})$$
    
$$MI = \sum\limits_{i}H(W_i\times X_{pca_{i}}) - H(X_{pca}) + \log_2 |W|$$


Поскольку $ W $ является матрицей вращения, ее определитель равен 1, поэтому $ \log_2 |W| = 0 $. Теперь $ H (X_ {pca}) $ не зависит от $ W $, поэтому мы можем отбросить его:


$$MI =  \sum\limits_{i}(H(W_i\times X_{pca_{i}}))$$


Таким образом,  


$$W = \arg\min_{W}\sum\limits_{i}(H(W_i\times X_{pca_{i}}))$$
    

Есть еще один способ найти матрицу $W_{negentropy} $, которая определяется выражением 

$$J(y) = H(y_g) - H(y)$$
    

Где $ H $ - энтропия, $ y $ - наш $X_ {pca}$, а $ x_ {gauss} $ - гауссовский случайный вектор с той же ковариацией, что и $ y $.
      
Среди всех распределений распределение Гауссово, как известно имеет наибольшую энтропию. Негентропия равна нулю, если $ y $ гауссова, и ненулевая, когда $ y $ не гауссов. Мы можем максимизировать функцию негэнтропии, чтобы получить матрицу $ W $.

При этом, $MI = константа - J (y)$.
    
Таким образом, оба метода сводятся к расчету энтропии, для которого нам нужно знать плотность распределения неизвестных переменных.

Оценивать распределения задача неблагодарная, плюс численно сложная, поэтому нужно делать аппроксимации.

Как мы говорили выше, одна из возможных аппроксимаций выражается через куртозис и третий момент, но такое приближение не очень робастное.

__(Hyvärinen, 1998b)__ предложил более хороший способ оценки негэнтропии:

$$J(y) = \sum\limits_{i=1}^{p} k_i \left[ \mathbb E G_i(y) - \mathbb E G_i(\mathcal N(0, 1))\right]^2, $$

*1. Заметим, что если $G = y^4$, то это превращается в формулу с куртозисом, т.е. это обобщение более частного уравнения.*

*2. Заметим, что второй член не зависит от данных, поэтому далее мы его опустим*

Где $g_i$ — некоторые хорошие функции. Примеры хороших функций:

$$G_1(y) =  \tanh(ay)$$

$$G_2(y) = -\exp(-y^2/2)$$

$$G_3(y) = -y \exp(-y^2/2)$$

$$G_4(y) = \log \cosh a_1 y$$

#### Псевдокод ICA

http://fourier.eng.hmc.edu/e161/lectures/ica/node7.html

Напомню, что мы ищем матрицу $W$, для следующего линейного преобразования данных $X$ в декоррелированные данные $Y$: 

$$Y = W X$$

Выписываем лагранжиан с использованием теоремы Каруша-Куна-Таккера(для обеспечения нормировки векторов $w$):

$$\mathcal{L} = \mathbb{E} G(w^T x) - \beta(w^T w - 1) / 2$$

Теперь мы можем свести задачу к задаче оптимизации функционала методом Ньютона(градиентный метод второго порядка). Тогда после нехитрых шагов выписывания производных, мы получаем красивую формулу для обновления текущей оценки W:


$$W = \mathbb E\{ XG(W^T X) \} - \mathbb E\{g(W^T X)\}W$$

$$W = \frac{W}{||W||}$$
 
где $G$ - аппроксимирующая функция, а $g$ - производная от $G$.

Прочитать про имплементацию можно [здесь](https://www.cs.helsinki.fi/u/ahyvarin/papers/bookfinal_ICA.pdf).

Псевдокод для нахождения W матриц:

1.$\hspace{1em}$ for $i$ in (1, number of sources):
2.    $\hspace{1em}W_i$ = random vector (initialization of  $W_i$) <p>
    $\hspace{1em}$while $W$ changes: 
4. $\hspace{2em}W_i$ = $\mathbb E\{XG($$W_i^T$$ X)\} - \mathbb E\{g(W_i^T X\}W_i\hspace{2em}$ (update of w)
5.    $\hspace{2em}W_i$ = $W_i$ - $\sum\limits_{j=0}^{i-1}W_iW_jW_j$   $\hspace{1em}$(orthogonalization of $W_i$ w.r.t other vectors of $W$ using gram-schmidt process)
6.    $\hspace{2em}W = W/||W||$
 
Таким образом, мы получаем $W_i$ ортогональную всем  $W$ векторам.  

In [ ]:
# approximating function is G(x) = x \exp(-x**2 / 2)
def g(x):
    return <YOUR cosine>

def gprime(x):
    return <YOUR cosine derivative>


# update of Wi
def update_vector(W, X):
    Y1 = W.dot(X)
    f1 =  (X * g(Y1)).mean(axis = 1)
    f2 = gprime(Y1).mean() * W
    W = f1 - f2
    return(W)

# orthonormalize the Wi
def orthonormalize(W, w, k):
    C= np.zeros_like(w)
    
    for j in range(k):
        C = C + np.dot(w, W[j].T) * W[j]
        
    w = w - C
    return(w / np.sqrt((w * w).sum()))

# calculation of unmixing matrix
# X.shape = number_of_dimensions, number of samples
def calculate_V(X, W_init):
    n_f, n_sample = X.shape
    W = np.zeros((n_f, n_f))
    
    for i in range(n_f):
        w = W_init[i, :].copy()
        w /= np.sqrt((w**2).sum())
        w_old = np.zeros((1, n_f))
        j=0
        max_iter = 1000
        while (np.abs(np.abs((w*w_old).sum())-1) > 1e-3 and (j < max_iter-1)):
            w_old = w
            w = update_vector(w, X)
            w = orthonormalize(W, w, i)
            j +=1
        W[i, :] = w
    return(W)

In [ ]:
# selection of number of components
# W = unmixing matrix
# S = source signal matrix
def ICA(X, n_comp=None):
    
    X = (X - X.mean(axis = 0))/(X.std(axis=0))
    pca = PCA(whiten=True)
    X_pca = pca.fit_transform(X)
    explained_var = pca.explained_variance_
    if not n_comp:
        n_comp = np.argmax(np.cumsum(explained_var)/explained_var.sum() > 0.99)+1
    K = pca.components_[0:n_comp]/pca.singular_values_
    X_pca = X_pca[:,0:n_comp]
    W_init = np.random.rand(n_comp, n_comp)
    V = calculate_V(X_pca.T, W_init)
    S = V.dot(X_pca.T)
    
    W = V.dot(K)
    
    return(W, S,V,K)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
np.random.seed(42)
num_rows = 3000
n_samples=1500
t = np.linspace(0,10, n_samples)
# create signals sources
s1 = np.sin(3*t) # a sine wave
s2 = np.sign(np.cos(6*t)) # a square wave
s3 = signal.sawtooth(2 *t) # a sawtooth wave
# combine single sources to create a numpy matrix
S = np.c_[s1,s2,s3]

# add a bit of random noise to each value
S += 0.2 * np.random.normal(size = S.shape)

# create a mixing matrix A
A = np.array([[1, 1.5, 0.5], [2.5, 1.0, 2.0], [1.0, 0.5, 4.0]])
X = S.dot(A.T)

#plot the single sources and mixed signals
plt.figure(figsize =(18, 8))
colors = ['red', 'blue', 'orange']

plt.subplot(2,1,1)
plt.title('True Sources')
for color, series in zip(colors, S.T):
    plt.plot(series, color)
plt.subplot(2,1,2)
plt.title('Observations(mixed signal)')
for color, series in zip(colors, X.T):
    plt.plot(series, color)

In [ ]:
# Теперь разложим на независимые компоненты наш сигнал:
W_, S_,V_ , K_= ICA(X, 3)
plt.figure(figsize =(18, 8))
plt.title('Recovered signals')
for color, series in zip(colors, S_):
    plt.plot(series, color)
plt.show()

##### Вопрос 3: Откуда мы знали, что нужно раскладывать на 3 компоненты?

СМЕКАЛОЧКА! Мы угодали сколько и какие компоненты выбрать.

ICA ничего нам не говорит о порядке компонентов или о том, сколько из них имеет значение. Таким образом, найти правильное количество компонент сложно.

* Мы можем ограничить количество компонентов с помощью PCA(по уровню explained variance первых компонент), затем найти такое же количество независимых компонентов;
* Мы можем оценить внутренную размерность сэмпла, допустим с MLE.

In [ ]:
"""
Implementation of 'Maximum Likelihood Estimation of Intrinsic Dimension' by Elizaveta Levina and Peter J. Bickel
 
how to use
----------
 
The goal is to estimate intrinsic dimensionality of data, 
the estimation of dimensionality is scale dependent
(depending on how much you zoom into the data distribution
you can find different dimesionality), so they
propose to average it over different scales, 
the interval of the scales [k1, k2] are the only parameters of the algorithm.
 
This code also provides a way to repeat the estimation with bootstrapping to estimate uncertainty.
 
Here is one example with swiss roll :
 
from sklearn.datasets import make_swiss_roll
X, _ = make_swiss_roll(1000)
 
k1 = 10 # start of interval(included)
k2 = 20 # end of interval(included)
intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             X, 
                             mode='bootstrap', 
                             nb_iter=500, # nb_iter for bootstrapping
                             verbose=1, 
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)
# the shape of intdim_k_repeated is (nb_iter, size_of_interval) where 
# nb_iter is number of bootstrap iterations (here 500) and size_of_interval
# is (k2 - k1 + 1).
 
# Plotting the histogram of intrinsic dimensionality estimations repeated over
# nb_iter experiments
plt.hist(intdim_k_repeated.mean(axis=1))
 
"""
# from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
 
def intrinsic_dim_sample_wise(X, k=5):
    neighb = <YOUR CODE>
    dist, ind = neighb.kneighbors(X) # distances between the samples and points
    dist = dist[:, 1:] # the distance between the first points to first points (as basis ) equals zero
    # the first non trivial point
    dist = dist[:, 0:k]# including points k-1
    assert dist.shape == (X.shape[0], k) # requirments are there is no equal points
    assert np.all(dist > 0)
    d = np.log(dist[:, k - 1: k] / dist[:, 0:k-1]) # dinstanec betveen the bayeasan statistics
    d = d.sum(axis=1) / (k - 2)
    d = 1. / d
    intdim_sample = d
    return intdim_sample
 
def intrinsic_dim_scale_interval(X, k1=10, k2=20):
    X = pd.DataFrame(X).drop_duplicates().values # remove duplicates in case you use bootstrapping
    intdim_k = []
    for k in range(k1, k2 + 1): # in order to reduse the noise by eliminating of the nearest neibours 
        m = intrinsic_dim_sample_wise(X, k).mean()
        intdim_k.append(m)
    return intdim_k
 
def repeated(func, X, nb_iter=100, random_state=None, mode='bootstrap', **func_kw):
    if random_state is None:
        rng = np.random
    else:
        rng = np.random.RandomState(random_state)
    nb_examples = X.shape[0]
    results = []
 
    iters = range(nb_iter) 
    for i in iters:
        if mode == 'bootstrap':# and each point we want to resample with repeating points to reduse the errors 
            #232 111 133 
            Xr = X[rng.randint(0, nb_examples, size=nb_examples)]
        elif mode == 'shuffle':
            ind = np.arange(nb_examples)
            rng.shuffle(ind)
            Xr = X[ind]
        elif mode == 'same':
            Xr = X
        else:
            raise ValueError('unknown mode : {}'.format(mode))
        results.append(func(Xr, **func_kw))
    return results

In [ ]:
k1 = 2 # start of interval(included)
k2 = 20 # end of interval(included)
nb_iter = 20 # more iterations more accuracy
# intrinsic_dim_scale_interval gives better estimation
intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             X, 
                             mode='bootstrap', 
                             nb_iter=nb_iter, # nb_iter for bootstrapping
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)


print (np.shape(intdim_k_repeated))

In [ ]:
x = np.arange(k1, k2+1)

plt.figure(figsize=(12, 8))
plt.plot(x, np.mean(intdim_k_repeated, axis=0), 'b', label='Mean') # it is the mean walue
plt.fill_between(x, 
                 np.mean(intdim_k_repeated, axis=0) - 
                 2 * np.std(intdim_k_repeated, axis=0),
                 np.mean(intdim_k_repeated, axis=0) + 
                 2 * np.std(intdim_k_repeated, axis=0),
                 alpha=0.3,
                 label='CI=95%',
                 color='g')
plt.legend()
plt.show()

# 1. Сравнение реализаций ICA

Кратко про метод: 

https://drive.google.com/viewerng/viewer?url=http://cs229.stanford.edu/notes/cs229-notes11.pdf 

In [ ]:
from sklearn.decomposition import PCA, FastICA
from numpy import dot

In [ ]:
# Генерируем сэмпл
rng = np.random.RandomState(42)
S = rng.standard_t(1.5, size=(20000, 2))
S[:, 0] *= 2.

# Mix data
A = np.array([[1, 1], [0, 2]])  # Mixing matrix

X = np.dot(S, A.T)  # Generate observations
X_mean = X.mean()
X = X - X_mean #shift mean

In [ ]:
# PCA
pca = PCA()
S_pca_ = pca.fit(X).transform(X)
S_pca_ /= S_pca_.std(axis=0)

In [ ]:
# ICA
ica = FastICA(random_state=rng)
S_ica_ = ica.fit(X).transform(X)  # Estimate the sources
S_ica_ /= S_ica_.std(axis=0)

In [ ]:
def np_sig(x):return  1 /(1 + (np.exp(-x)))
def np_tanh(x): return np.tanh(x)

num_epoch = 10000
learning_rate = 0.00003

In [ ]:
# ICA со спуском по сигмоиде
# Читать здесь http://www.sci.utah.edu/~shireen/pdfs/tutorials/Elhabian_ICA09.pdf
w_sig = np.eye(2)
for iter in range(num_epoch):
    temp = np_sig(dot(X, w_sig))
    temp = 1-2*temp
    w_sig = w_sig + learning_rate * (np.linalg.inv(w_sig.T) + dot(temp.T, X))

S_sig_grad = dot(X,w_sig)
S_sig_grad /= S_sig_grad.std(axis=0)

In [ ]:
# ICA со спуском по tanh
# Elhabian method
# Читать здесь http://www.sci.utah.edu/~shireen/pdfs/tutorials/Elhabian_ICA09.pdf
w_tanh_grad = np.eye(2)
for current_iter in range(num_epoch):
    u = dot(X,w_tanh_grad)
    U = np_tanh(u)
    g = np.linalg.inv(w_tanh_grad.T) - (2 / len(X)) * dot(X.T, U)
    w_tanh_grad = w_tanh_grad + learning_rate * g

S_tanh_grad = dot(X,w_tanh_grad)
S_tanh_grad /= S_tanh_grad.std(axis=0)

In [ ]:
print('Original Data Shape',S.shape)
print('Fast ICA Data Shape',S_ica_.shape)
print('PCA Data Shape',S_pca_.shape)
print('Sigmoid Grad Data Shape',S_sig_grad.shape)
print('Tanh Grad Data Shape',S_tanh_grad.shape)

In [ ]:
def plot_samples(S, axis_list=None):
    plt.scatter(S[:, 0], S[:, 1], s=2, marker='o', zorder=10,
                color='steelblue', alpha=0.5)
    if axis_list is not None:
        colors = ['orange', 'red']
        for color, axis in zip(colors, axis_list):
            axis /= axis.std()
            x_axis, y_axis = axis
            # Trick to get legend to work
            plt.plot(0.1 * x_axis, 0.1 * y_axis, linewidth=2, color=color)
            plt.quiver(0, 0, x_axis, y_axis, zorder=11, width=0.01, scale=6,
                       color=color)

    plt.hlines(0, -3, 3)
    plt.vlines(0, -3, 3)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.xlabel('x')
    plt.ylabel('y')

plt.figure(figsize=(15,15))
plt.subplot(3, 2, 1)
plot_samples(S / S.std())
plt.title('True Independent Sources')

axis_list = [pca.components_.T, ica.mixing_]
plt.subplot(3, 2, 2)
plot_samples(X / np.std(X), axis_list=axis_list)
legend = plt.legend(['PCA', 'ICA'], loc='upper right')
legend.set_zorder(100)

plt.title('Observations')

plt.subplot(3, 2, 3)
plot_samples(S_pca_ / np.std(S_pca_, axis=0))
plt.title('PCA recovered signals')

plt.subplot(3, 2, 4)
plot_samples(S_ica_ / np.std(S_ica_))
plt.title('ICA recovered signals')

plt.subplot(3, 2, 5)
plot_samples(S_sig_grad / np.std(S_sig_grad, axis=0))
plt.title('ICA sig recovered signals')

plt.subplot(3, 2, 6)
plot_samples(S_tanh_grad / np.std(S_tanh_grad))
plt.title('ICA tanh recovered signals')

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.36)
plt.show()

FastICA и ICA с функцией tanh лучше всего справились с разложением.

Заметим, что по умолчанию sklearn `FastICA` использует $\log \cosh(y)$ функцию.

# 2. ICA для анализа fMRT данных

In [ ]:
from nilearn import datasets
# Here we use only 3 subjects to get faster-running code. For better
# results, simply increase this number
# XXX: must get the code to run for more than 1 subject
dataset = datasets.fetch_adhd(n_subjects=1)
func_filename = dataset.func[0]

# print basic information on the dataset
print('First subject functional nifti image (4D) is at: %s' %
      dataset.func[0])  # 4D data

In [ ]:
from nilearn.input_data import NiftiMasker

# This is resting-state data: the background has not been removed yet,
# thus we need to use mask_strategy='epi' to compute the mask from the
# EPI images
masker = NiftiMasker(smoothing_fwhm=8, memory='nilearn_cache', memory_level=1,
                     mask_strategy='epi', standardize=True)
data_masked = masker.fit_transform(func_filename)

# Concatenate all the subjects
# fmri_data = np.concatenate(data_masked, axis=1)
fmri_data = data_masked

In [ ]:
n_components = 20
ica = FastICA(n_components=n_components, random_state=42)
components_masked = ica.fit_transform(data_masked.T).T

# Normalize estimated components, for thresholding to make sense
components_masked -= components_masked.mean(axis=0)
components_masked /= components_masked.std(axis=0)
# Threshold
import numpy as np
components_masked[np.abs(components_masked) < .8] = 0

# Now invert the masking operation, going back to a full 3D
# representation
component_img = masker.inverse_transform(components_masked)

In [ ]:
# Show some interesting components
from nilearn import image
from nilearn.plotting import plot_stat_map, show

# Use the mean as a background
mean_img = image.mean_img(func_filename)

plot_stat_map(image.index_img(component_img, 0), mean_img)

plot_stat_map(image.index_img(component_img, 2), mean_img)

show()

Мы выбрали 10 первых компонент и построили 2 из них. А какая размерность данных?

In [ ]:
import sys
sys.setrecursionlimit(10000)

In [ ]:
k1 = 1 # start of interval(included)
k2 = 5 # end of interval(included)
nb_iter = 2 # more iterations more accuracy
# intrinsic_dim_scale_interval gives better estimation
intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             data_masked, 
                             mode='bootstrap', 
                             nb_iter=nb_iter, # nb_iter for bootstrapping
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)


print (np.shape(intdim_k_repeated))

In [ ]:
x = np.arange(k1, k2+1)

plt.figure(figsize=(12, 8))
plt.plot(x, np.mean(intdim_k_repeated, axis=0), 'b', label='Mean') # it is the mean walue
plt.fill_between(x, 
                 np.mean(intdim_k_repeated, axis=0) - 
                 2 * np.std(intdim_k_repeated, axis=0),
                 np.mean(intdim_k_repeated, axis=0) + 
                 2 * np.std(intdim_k_repeated, axis=0),
                 alpha=0.3,
                 label='CI=95%',
                 color='g')
plt.legend()
plt.show()

# 3. Вспомним задачу, которую мы не решили с PCA

In [ ]:
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from matplotlib import offsetbox

def plot_embedding(X, y, images_small=None, title=None):
    """
    Функция для рисования эмбеддингов 
    с цветами соответствующими y
    """
    # take only first two columns
    X = X[:, :2]
    # scaling
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)
    plt.figure(figsize=(13,8))
    ax = plt.subplot(111)
    for i in range(X.shape[0] - 1):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.RdGy(y[i]),
                 fontdict={'weight': 'bold', 'size': 12})
        if images_small is not None:
            imagebox = OffsetImage(images_small[i], zoom=.3)
            ab = AnnotationBbox(imagebox, (X[i, 0], X[i, 1]),
                xycoords='data')                                  
            ax.add_artist(ab)
    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-1:
                # don't show points that are too close
                continue
    if title is not None:
        plt.title(title)

In [ ]:
import pandas as pd
data = pd.read_csv('./data.csv')
mask = data.CDR.notnull()
data = data[mask]
data.head()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
def spectrum(x):
    f = np.fft.fft2(x)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.log(np.abs(fshift))
    return magnitude_spectrum

In [ ]:
brain_data = pd.read_csv('./brain_data.csv.zip')[mask]
brain_data.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
X_brain = np.log(brain_data.values / 255. + 1)
X_brain_image = X_brain.reshape((-1, 176, 176))
age = data.Age.values
cdr = (data.CDR.values == 0).astype(int)

In [ ]:
plt.imshow(X_brain_image[2], cmap='gray')
plt.show()

In [ ]:
from sklearn.decomposition import PCA, FastICA

In [ ]:
X_projected = FastICA(n_components=3, max_iter=1000).fit_transform(brain_data)

plot_embedding(X_projected, cdr, X_brain_image, "ICA. Pis.")
plot_embedding(X_projected, age, title="ICA. CDR.")
plot_embedding(X_projected, cdr, title="ICA. CDR.")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

scores = cross_val_score(estimator=XGBClassifier(), 
                         X=X_projected, 
                         y=LabelEncoder().fit_transform(cdr), 
                         cv=5,
                         scoring='roc_auc')

print('Средняя точность: {} +- {}'.format(1 - np.mean(scores), np.std(scores)))